In [1]:
import string

In [64]:
def verifica_digito(character : str):
    digitos = ['0','1','2','3','4','5','6','7','8','9']
    if character in digitos:
        return True
    else:
        return False

In [16]:
def verifica_se_letra(character : str):
    
    if (character in string.ascii_lowercase) or (character in string.ascii_uppercase):
        return True
    else:
        return False

In [53]:
def verifica_existencia_caractere(character : str):
    if (character != None) and not(character.isspace()):
        return True
    else:
        return False

In [68]:
palavras_reservadas = ['inicio', 'varinicio', 'varfim', 'escreva', 'leia', 
                      'se', 'entao', 'fimse', 'fim', 'inteiro', 'literal', 'real']

In [120]:
indice_tabela = {'D':0, '\"':1, 'L':2, '{':3}

In [121]:
indice_tabela['"']

1

In [122]:
                    #    D  "   L  {
tabela_de_estados = [  # 0  1   2  3
                        [1, 7, 10, 12], #0 
                        [], #1
                        [], #2
                        [], #3 
                        [], #4
                        [], #5
                        [], #6 
                        [], #7
                        [], #8
                        [], #9 
                        [], #10
                        [], #11
                        [], #12 
                        [], #13
                        [], #14
                        [], #15
                        [], #16
                        [], #17
                        [], #18
                        [], #19
                        [], #20
                        [], #21
                        [], #22
                        [], #23
                    ]

In [114]:
                #[estado atual] [ caractere recebido] return proximo estado
tabela_de_estados[0][0]

1

In [1]:
tabelaDeEstados = {

      0: {'D':1, '"':10, 'L':13, '{':14, '$':16, ',':26, ';':25, '(':23, ')':24, '+':22, '-':22, '*':22, '/':22, '=':21, '>':19, '<':17, 'E':13, ' ':0},       
      
      1: {'D':1, '.':2, 'E':5},

      2: {'D':3},      

      3: {'D':3, 'E':4},     

      4: {'D':11, '+':9, '-':9},

      5: {'D':8, '+':7, '-':6},

      6: {'D':11},

      7: {'D':8},

      8: {'D':8},    

      9: {'D':11},
      
      10: {'"':12, ',':10,';':10,':':10,'.':10,'!':10,'?':10,'\\':10,'*':10,'+':10,'-':10,'/':10,'(':10,')':10,'{':10,'}':10,'[':10,']':10,'<':10,'>':10,'=':10,'\'':10, ' ':10, 'D':10, 'L':10, 'E':10},

      13: {'D':13, 'L':13, 'E':13, '_':13},

      11: {'D':11},
      
      14: {'':14, '}':15, ':':14, '\\':14, ',':14,';':14,':':14,'.':14,'!':14,'?':14,'\\':14,'*':14,'+':14,'-':14,'/':14,'(':14,')':14,'{':14, '[':14,']':14,'<':14,'>':14,'=':14,'\'':14, ' ':14, 'D':14, 'L':14, 'E':14},
      
      17: {'-':18, '=':20, '>':20},
    }

In [4]:
tabelaDeEstados[6]['L']

KeyError: 'L'

In [123]:
def realiza_mudanca_de_estado(estado_atual, character : str):
    character_index = character

    if verifica_digito(character):
        character_index = 'D'

    if verifica_se_letra(character):
        character_index  = 'L'
        
    return tabela_de_estados[estado_atual][indice_tabela[character_index]]

In [139]:
realiza_mudanca_de_estado(0, 'A')

10

In [136]:
class Token():
    def __init__(self, classe : str, lexema : str, tipo : str) -> None:
        self.classe = classe
        self.lexema = lexema
        self.tipo = tipo
    
    def __str__(self) -> str:
        return f"Classe: {self.classe}, Lexema: {self.lexema}, Tipo: {self.tipo}"
    def __repr__(self) -> str:
        return f"Classe: {self.classe}, Lexema: {self.lexema}, Tipo: {self.tipo}"

In [137]:
x = Token(classe = "Qualquer merda", lexema = "Nem sei o que é", tipo = "2")

In [ ]:
tabela_de_simbolos = [x] #Uma lista de tokens